In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [119]:
#計測地点のosm_idを指定
start_osm = 668319081
mid_osm = [385509583]
end_osm = 385509577
start_mesh = 50325641322
end_mesh = 50325633332

In [120]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [121]:

sttime = "2022-04-01 12:00:00"
dbname = "kddi_trkn12_change_f_c23"

In [122]:
datedir = "230213"
f = open(datedir+'/'+ dbname + '.csv', 'w')

cur =connection.cursor()
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
cur.execute(sql)
for row in cur:
    stlat = row[0]
    stlng = row[1]
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
cur.execute(sql)
for row in cur:
    edlat = row[0]
    edlng = row[1]
cur.close()

if stlat > edlat:
    wklat = stlat
    stlat = edlat
    edlat = wklat
if stlng > edlng:
    wklng = stlng
    stlng = edlng
    edlng = wklng
            
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_12_param  "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)


        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        sql += "ORDER BY trakanmap.gridcode"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                #cur.execute(sql)
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + '\n')
        #cur.execute(sql)
        #connection.commit()
                
        cur.close()
        #print(lstItems)
        dte = dte + datetime.timedelta(minutes=15)
f.close()

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
存在メッシュ：50325641322値126.0距離0.0 km
50325641144
存在メッシュ：50325641144値127.61872487796072距離0.11554019701202542 km
50325642232
存在メッシュ：50325642232値146.53089137252772距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値149.2226269394562距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値144.31818070749483距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値142.21903025904174距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値148.8241649745856距離1.6291270701422988 km
50325642411
存在メッシュ：50325642411値144.246409038417距離1.3023792515727701 km
50325642214
存在メッシュ：50325642214値146.84763325407877距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値146.33842962750865距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値149.72506723221395距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値144.5316102821526距離1.3227361438019158 km
50325642321
存在メッシュ：50325642321値140.191651478697距離1.0129616404242632 km
50325633331
存在メッシュ：50325633331値1

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
存在メッシュ：50325641322値140.0距離0.0 km
50325641144
存在メッシュ：50325641144値142.80578978846523距離0.11554019701202542 km
50325642232
存在メッシュ：50325642232値175.58687837904804距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値180.2525533617241距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値171.7515132263244距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値168.11298578233902距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値179.5618859559484距離1.6291270701422988 km
50325642411
存在メッシュ：50325642411値171.62710899992283距離1.3023792515727701 km
50325642214
存在メッシュ：50325642214値176.1358976404032距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値175.25327802101498距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値181.12344986917083距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値172.12145782239782距離1.3227361438019158 km
50325642321
存在メッシュ：50325642321値164.5988625630748距離1.0129616404242632 km
50325633331
存在メッシュ：50325633331

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
存在メッシュ：50325641322値133.0距離0.0 km
50325641144
存在メッシュ：50325641144値135.32017232507704距離0.11554019701202542 km
50325642232
存在メッシュ：50325642232値162.42761096728972距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値166.28576527988724距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値159.25605901407596距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値156.2472767046265距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値165.71463646357273距離1.6291270701422988 km
50325642411
50325642411 is kddi none!
50325642214
存在メッシュ：50325642214値162.88160766417957距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値162.15174913276238距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値167.00592969950665距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値159.56197473775205距離1.3227361438019158 km
50325642321
存在メッシュ：50325642321値153.3413671194657距離1.0129616404242632 km
50325633331
存在メッシュ：50325633331値173.26148349312405距離2.004945789

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
50325641322 is kddi none!
50325641144
存在メッシュ：50325641144値118.88851235762084距離0.11554019701202542 km
50325642232
存在メッシュ：50325642232値140.95270660128233距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値144.0930647626989距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値138.37121082541066距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値135.92220196888204距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値143.6281924703499距離1.6291270701422988 km
50325642411
存在メッシュ：50325642411値138.2874772114865距離1.3023792515727701 km
50325642214
存在メッシュ：50325642214値141.3222387964252距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値140.72816789876006距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値144.67924510424962距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値138.62021199584467距離1.3227361438019158 km
50325642321
存在メッシュ：50325642321値133.5569267251465距離1.0129616404242632 km
50325633331
存在メッシュ：50325633331値149.77

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
存在メッシュ：50325641322値135.0距離0.0 km
50325641144
存在メッシュ：50325641144値137.4280873169411距離0.11554019701202542 km
50325642232
存在メッシュ：50325642232値165.79633705879158距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値169.83394040918432距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値162.47727106124228距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値159.32854538856262距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値169.23624746187843距離1.6291270701422988 km
50325642411
存在メッシュ：50325642411値162.3696135576255距離1.3023792515727701 km
50325642214
存在メッシュ：50325642214値166.27144988111814距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値165.50764444126295距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値170.5876008483209距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値162.79741542322887距離1.3227361438019158 km
50325642321
50325642321 is kddi none!
50325633331
存在メッシュ：50325633331値177.13411063233912距離2.0049457898

...

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
存在メッシュ：50325641322値98.0距離0.0 km
50325641144
存在メッシュ：50325641144値99.88851235762084距離0.11554019701202542 km
50325642232
存在メッシュ：50325642232値121.95270660128234距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値125.09306476269892距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値119.37121082541066距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値116.92220196888204距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値124.62819247034989距離1.6291270701422988 km
50325642411
50325642411 is kddi none!
50325642214
存在メッシュ：50325642214値122.32223879642522距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値121.72816789876008距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値125.6792451042496距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値119.62021199584468距離1.3227361438019158 km
50325642321
存在メッシュ：50325642321値114.5569267251465距離1.0129616404242632 km
50325633331
存在メッシュ：50325633331値130.77097493626374距離2.00494578980

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
50325641322 is kddi none!
50325641144
50325641144 is kddi none!
50325642232
存在メッシュ：50325642232値123.68726091501848距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値125.48175129297081距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値122.21212047166323距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値120.81268683936116距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値125.21610998305708距離1.6291270701422988 km
50325642411
存在メッシュ：50325642411値122.164272692278距離1.3023792515727701 km
50325642214
存在メッシュ：50325642214値123.89842216938584距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値123.55895308500575距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値125.81671148814263距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値122.35440685476838距離1.3227361438019158 km
50325642321
50325642321 is kddi none!
50325633331
存在メッシュ：50325633331値128.72627139215072距離2.004945789807886 km
50325633332
存在メッシュ：50325633332値13

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
存在メッシュ：50325641322値113.0距離0.0 km
50325641144
50325641144 is kddi none!
50325642232
存在メッシュ：50325642232値125.31853482351663距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値126.93357616367372距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値123.99090842449691距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値122.73141815542505距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値126.69449898475138距離1.6291270701422988 km
50325642411
存在メッシュ：50325642411値123.94784542305021距離1.3023792515727701 km
50325642214
存在メッシュ：50325642214値125.50857995244726距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値125.20305777650518距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値127.23504033932836距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値124.11896616929155距離1.3227361438019158 km
50325642321
50325642321 is kddi none!
50325633331
存在メッシュ：50325633331値129.85364425293565距離2.004945789807886 km
50325633332
存在メッシュ：50325

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
存在メッシュ：50325641322値137.0距離0.0 km
50325641144
50325641144 is kddi none!
50325642232
存在メッシュ：50325642232値165.0588848757879距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値168.73759015059017距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値162.0348469669096距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値159.16600802069038距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値168.19302546526703距離1.6291270701422988 km
50325642411
存在メッシュ：50325642411値161.9367590191699距離1.3023792515727701 km
50325642214
存在メッシュ：50325642214値165.49176544724097距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値164.7958538242618距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値169.4242585506924距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値162.3265340522752距離1.3227361438019158 km
50325642321
存在メッシュ：50325642321値156.3952570208859距離1.0129616404242632 km
50325633331
存在メッシュ：50325633331値175.38885635390898距離2.004945789807886

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322
存在メッシュ：50325641322値127.0距離0.0 km
50325641144
存在メッシュ：50325641144値128.51080988609667距離0.11554019701202542 km
50325642232
存在メッシュ：50325642232値146.16216528102586距離1.4654394124113244 km
50325642221
存在メッシュ：50325642221値148.67445181015913距離1.657568205847179 km
50325642233
存在メッシュ：50325642233値144.0969686603285距離1.3075021188975637 km
50325642322
存在メッシュ：50325642322値142.13776157510563距離1.1576704460331162 km
50325642223
存在メッシュ：50325642223値148.3025539762799距離1.6291270701422988 km
50325642411
50325642411 is kddi none!
50325642214
存在メッシュ：50325642214値146.45779103714017距離1.488047589930948 km
50325642234
存在メッシュ：50325642234値145.98253431900807距離1.4517020143892478 km
50325632443
存在メッシュ：50325632443値149.1433960833997距離1.693431032941764 km
50325642231
存在メッシュ：50325642231値144.29616959667575距離1.3227361438019158 km
50325642321
存在メッシュ：50325642321値140.2455413801172距離1.0129616404242632 km
50325633331
存在メッシュ：50325633331値153.216779949011距離2.0049457898078

In [123]:
connection.close()

In [63]:
#CreateGeojsonFile